## Loading and preparing the data
### Loading the Excel data
The concrete strength data set is saved in an Excel Workbook. We can use the `read_excel` function of the `pandas` library to load the data as a dataframe.

In [ ]:
import pandas as pd
data = pd.read_excel("Concrete_Data.xls")

By calling the "head" method of the loaded dataframe, we can examine the structure with regard to the first rows.

In [ ]:
data.head()

### Splitting the data by explanatory variables and outcome variable
The first eight columns contain our explanatory variables which we want to use to **predict the outcome for concrete strength** (dependent variable), listed in the last column. To use this data as part of our regression models, we need to split it into the data on **independent variables** and the data on the **outcome variable**.

For this purpose, we utilize the `iloc` method which allows us to slice rows and columns by their integer position, allowing selection of specific ranges or individual elements based on their index location. The first part of the argument before the comma indicates we want to select all rows whereas the second part indicates column selection. For our X data, we select all columns excluding the last column (half-open range). For our Y data, we select only the last column.

In [ ]:
X, Y = data.iloc[:, 0:8], data.iloc[:, 8]

### Creating a train-test split
To obtain a fair evaluate of the model's performance, we split the observations into training and test data. We use the latter to evaluate the trained model on unseen data.

We can easily split the data using the `train_test_split` function from the `sklearn` library. The function splits the rows according to our specification of the relative share of training and test data.

Here, we use 80% of our observations as training data and 20% for evaluation purposes.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.80, test_size=0.20)

### Normalizing the data of the dependent variables
We normalize the features to ensure all data is on a similar scale. We can use the `MinMaxScaler` for this task which automatically rescales our feature data.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Linear Regression Model
First, we fit a linear regression model as a baseline. We use the `keras` library which provides an easy way to build an run machine learning models in Python 3.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

### Building the linear regression model in `keras`
The following code sets up a `Sequential` model for keras. We add a single layer where all explanatory variables (X) are directly connected to the single outcome variable (Y), modeling the typical structure of a linear regression model. 

Note that we use `keras` for the linear regression model and machine learning model to achieve consistency. Typically, you would fit the regression model using ordinary least squares. For our purpose here, the result will be the same.

# INSERT IMAGE OF NETWORK STRUCTURE FROM LECTURE HERE!

In [ ]:
linear_regression = Sequential()
linear_regression.add(Dense(1, activation='linear'))
linear_regression.compile(optimizer="SGD", loss='mean_squared_error')

### Fitting the linear regression model
To start the training and fit our model, we simply call the `fit` method, specifying the data as well as the number of iterations (epochs).

In [ ]:
linear_regression.fit(X_train, Y_train, epochs=1000, verbose=0)

# Deep Neural Network Model

### Building the deep neural network in `keras`
The following code sets up a Sequential model for keras. We first add a dense layer with 64 neurons and ReLU activation, followed by another dense layer with 32 neurons and ReLU activation, modeling a deep neural network structure. The final layer is a single neuron with linear activation to produce a continuous output for the regression task.

We compile the model using the Adam optimizer and mean squared error loss function to fit the neural network.

In [ ]:
neural_network = Sequential()
neural_network.add(Dense(64, activation='relu'))
neural_network.add(Dense(32, activation='relu'))
neural_network.add(Dense(1, activation='linear'))
neural_network.compile(optimizer='adam', loss='mean_squared_error')

### Fitting the deep neural network model
Next, we fit the neural network by calling the `fit` method. The batch size specifies the number of observations processed before updating the model parameters.

In [ ]:
neural_network.fit(X_train, Y_train, epochs=1000, batch_size=10)

## Evaluating the model performance
For this purpose, we will utilize the following measures:

### Mean Squared Error (MSE)
$\text{MSE} = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2$

### R-squared (R²)
$R^2 = 1 - \frac{\sum_{i=1}^{n} (y_i - \hat{y}_i)^2}{\sum_{i=1}^{n} (y_i - \bar{y})^2}$


First, we apply each of the fitted models to the test set to obtain predictions for the concrete strength.

In [ ]:
Y_lr = linear_regression.predict(X_test).flatten()
Y_nn = neural_network.predict(X_test).flatten()

Then, we can use the predictions to calculate the performance measures based on the predictions and actual concrete strength observations.

In [ ]:
mse_lr = ((Y_test - Y_lr) ** 2).mean()
r2_lr = 1 - mse_lr / ((Y_test - Y_test.mean()) ** 2).mean()

mse_nn = ((Y_test - Y_nn) ** 2).mean()
r2_nn = 1 - mse_nn / ((Y_test - Y_test.mean()) ** 2).mean()

Looking at the performance metrics, it becomes clear that the neural network model is able to explain a much higher fraction of the variation in concrete strength based on the features.

In [ ]:
print(f"{'Metric':<25} {'Linear Regression':<20} {'Neural Network'}")
print(f"{'Mean Squared Error':<25} {mse_lr:<20} {mse_nn}")
print(f"{'R-squared':<25} {r2_lr:<20} {r2_nn}")

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(14, 6))

# Linear Regression
plt.subplot(1, 2, 1)
plt.scatter(Y_test, Y_lr, alpha=0.6)
plt.plot([Y_test.min(), Y_test.max()], [Y_test.min(), Y_test.max()], 'r--')
plt.xlabel('Actual concrete compressive strength (MPa)')
plt.ylabel('Predicted concrete compressive strength (MPa)')
plt.title('Linear Regression')

# Neural Network
plt.subplot(1, 2, 2)
plt.scatter(Y_test, Y_nn, alpha=0.6)
plt.plot([Y_test.min(), Y_test.max()], [Y_test.min(), Y_test.max()], 'r--')
plt.xlabel('Actual concrete compressive strength (MPa)')
plt.ylabel('Predicted concrete compressive strength (MPa)')
plt.title('Neural Network')
plt.savefig("fit.pdf")